# Pokemon Match

This file calculates the top x number of attacking pokemon per type for raids (PvE). It lists:
- the top moves for each type
- a summary of the pokemon for each type (split by normal/shadow) 
- a combined summary of all pokemon of all types at the end (split norm/shadow)

Use the "Runtime -> Run All" to see all output :)

In [1]:
import os
import shutil
print(os.getcwd())

/content


In [2]:
# this bit imports the github, clones and changes the path:
os.chdir('/content')
myDir = './drmckenzie.github.io/'

# WARNING! this will delete any changes to the files you have made
if os.path.isdir(myDir):
    shutil.rmtree(myDir)

!git clone https://github.com/drmckenzie/drmckenzie.github.io.git

os.chdir(myDir)
print(os.getcwd())

Cloning into 'drmckenzie.github.io'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 176 (delta 107), reused 127 (delta 67), pack-reused 0
Receiving objects: 100% (176/176), 1.12 MiB | 2.45 MiB/s, done.
Resolving deltas: 100% (107/107), done.
/content/drmckenzie.github.io


# You can change these default settings to customise the results:

In [3]:
# This is the list of types that runs by default:
typeList = ["Normal", "Fire", "Water", "Grass", "Flying", "Fighting", "Poison", "Electric", "Ground", "Rock", "Psychic", "Ice", "Bug", "Ghost", "Steel", "Dragon", "Dark", "Fairy"]

# This is number of pokemon returned per type.
findThisMany = 5


In [4]:
import pandas as pd
 
import pokemonMatch as match
 
runCases =  'basic'
 
whatFlag = runCases
 
#flag for running cases
#whatFlag = 'both' # 'basic' 'shadow' 'both'
 
# GamePress total CPO filename
gamepressFilenameShadow = 'shadow_comprehensive_dps_gamepress_L35_best_moves.csv'
gamepressFilename = 'basic_comprehensive_dps_gamepress_L35_all_moves.csv'
gamepressFilename = 'all_comprehensive_dps_gamepress_L35_all_moves.csv'
#gamepressFilename = 'mega_comprehensive_dps_gamepress_L35_all_moves.csv'
 
# setting the filters
typeFilter = []
topX = 0
 
# load data
allData = match.getGamepressData(gamepressFilename,typeFilter,topX)
    
# print(list(allData.columns))
 
# type lookup:
allData = match.typeAttackLookup(allData)
 
allData["Shadow"]=False
 
name = allData["Pokemon"]
nameloc = name.str.startswith("Shadow")
allData.loc[nameloc,"Shadow"]=True

# add a tag to the existing data, to say it's top tier
allData["TopPvE"] = False
 
# finding Pokémon with both attack types the same
#typeList = ["Normal", "Fire", "Water", "Grass", "Flying", "Fighting", "Poison", "Electric", "Ground", "Rock", "Psychic", "Ice", "Bug", "Ghost", "Steel", "Dragon", "Dark", "Fairy"]
 
setData = pd.DataFrame() 
 
#findThisMany = 5
 
# Setting empty strings for later
joinedStrings = ""
joinedStringsShad = ""
joinedStringsAll = ""
# setting a list of empty, for listing all the top x later
joinedIntBase = list()
joinedIntShad = list()
 
# big loop now:
for thisType in typeList:
    setDataTemp = match.findBothAttackOfType(allData,thisType)
    setData = pd.concat([setData,setDataTemp.head(findThisMany)],sort=False)
    topTypeSet = setDataTemp[["Pokemon","#","DPS^3*TDO","Fast move","Special move","TypeFast","Shadow"]]
    topTypeSetPrint = setDataTemp[["Pokemon","DPS^3*TDO","Fast move","Special move","TypeFast"]]
    topTypeSetPrint = topTypeSetPrint.iloc[:findThisMany]
    print(topTypeSetPrint.to_string(index=False))
    
    print(" ")
    
    # select top X
    topTypeSet = topTypeSet.head(findThisMany)
 
    # separate Shadow from non-Shadow
    shadTop = topTypeSet[topTypeSet["Shadow"]==True]
    shadTopNot = topTypeSet[topTypeSet["Shadow"]==False]
 
    #print(shadTopNot.head(5))
 
    #print(len(shadTopNot))
 
    if len(shadTopNot) > 0:
        # base
        shadString = ""
        typeListTemp = shadTopNot["#"].head(findThisMany)
        joinedIntBase.extend(typeListTemp)
        typeStr = [str(int(inte)) for inte in typeListTemp]
        typeStr = ",".join(typeStr)
        #print(typeStr)
        allString = "Type: "+thisType+" top "+str(findThisMany)+" - "+typeStr+shadString
    else:
        allString = "Type: "+thisType+" not found in base"
    print(allString) 
 
    if len(shadTop) > 0:
        # shadow
        shadString = "&shadow"
        typeListTemp = shadTop["#"].head(findThisMany)
        joinedIntShad.extend(typeListTemp)
        typeStr = [str(int(inte)) for inte in typeListTemp]
        typeStr = ",".join(typeStr)
        #print(typeStr)
        allStringShad = "Type: Shadow "+thisType+" top "+str(findThisMany)+" - "+typeStr+shadString
    else:
        allStringShad = "Type: "+thisType+" not found in shadow"
    print(allStringShad)  
 
    joinedStrings = joinedStrings + "\n" + allString
    joinedStringsShad = joinedStringsShad + "\n" + allStringShad
 
    joinedStringsAll = joinedStringsAll + "\n" + allString + "\n" + allStringShad
 
    print(" ")
 
print("------------------------------------")
print("This is a summary list of pokemon per types")
 
print(joinedStrings)
 
print(joinedStringsShad)
 
print(joinedStringsAll)
 
print("")
print("------------------------------------")
print("This is a summary list of all pokemon gathered together of all types")
 
# make the lists sorted and unique:
joinedIntBase = list(sorted(set(joinedIntBase)))
joinedIntShad = list(sorted(set(joinedIntShad)))
 
typeStr = [str(int(inte)) for inte in joinedIntBase]
typeStr = ",".join(typeStr)
print("List of combined top "+str(findThisMany)+" base pokemon of all types: ")
print(typeStr)

print("")
 
typeStr = [str(int(inte)) for inte in joinedIntShad]
typeStr = ",".join(typeStr)
print("List of combined top "+str(findThisMany)+" Shadow pokemon of all types: ")
print(typeStr+"&shadow")
 
# mark this set as top tier for the future
setData["TopPvE"] = True
 
#print(allData[allData["Pokemon"]=="Kyogre"])
 
#print(allData.head(20))

          Pokemon  DPS^3*TDO Fast move Special move TypeFast
 Shadow Porygon-Z     1781.3   Lock-On   Hyper Beam   Normal
        Porygon-Z     1255.1   Lock-On       Return   Normal
   Shadow Ambipom      807.2   Scratch   Hyper Beam   Normal
  Shadow Porygon2      769.7   Lock-On   Hyper Beam   Normal
          Ambipom      599.5   Scratch       Return   Normal
 
Type: Normal top 5 - 474,424
Type: Shadow Normal top 5 - 474,424,233&shadow
 
          Pokemon  DPS^3*TDO  Fast move  Special move TypeFast
     Shadow Entei     3031.7  Fire Fang      Overheat     Fire
         Reshiram     2943.1  Fire Fang      Overheat     Fire
   Shadow Moltres     2916.5  Fire Spin      Overheat     Fire
 Shadow Charizard     2421.3  Fire Spin    Blast Burn     Fire
  Shadow Arcanine     1824.6  Fire Fang  Flamethrower     Fire
 
Type: Fire top 5 - 643
Type: Shadow Fire top 5 - 244,146,6,59&shadow
 
         Pokemon  DPS^3*TDO  Fast move  Special move TypeFast
 Shadow Swampert     2747.5  Water Gun  H

#TODO - instead of doing just the top move, do 95% or 90% is acceptable???

In [5]:
print(allData.loc[1,"Shadow"]==False)
setData.head(25)

True


,Pokemon,Fast move,Special move,DPS,TDO,DPS^3*TDO,Special move 2,#,Type1,Type2,Released_2021_03_13,TypeFast,TypeSpecial,Shadow,TopPvE
674,Shadow Porygon-Z,Lock-On,Hyper Beam,17.528,330.8,1781.3,Hyper Beam,474.0,Normal,NaN,True,Normal,Normal,True,True
2129,Porygon-Z,Lock-On,Return,15.343,347.5,1255.1,Return,474.0,Normal,NaN,True,Normal,Normal,False,True
705,Shadow Ambipom,Scratch,Hyper Beam,14.843,246.8,807.2,Hyper Beam,424.0,Normal,NaN,True,Normal,Normal,True,True
676,Shadow Porygon2,Lock-On,Hyper Beam,13.629,304.0,769.7,Hyper Beam,233.0,Normal,NaN,True,Normal,Normal,True,True
2246,Ambipom,Scratch,Return,13.165,262.7,599.5,Return,424.0,Normal,NaN,True,Normal,Normal,False,True
2284,Shadow Entei,Fire Fang,Overheat,18.379,488.3,3031.7,Overheat,244.0,Fire,NaN,True,Fire,Fire,True,True
2285,Reshiram,Fire Fang,Overheat,17.409,557.8,2943.1,Overheat,643.0,Dragon,Fire,True,Fire,Fire,False,True
2289,Shadow Moltres,Fire Spin,Overheat,18.796,439.2,2916.5,Overheat,146.0,Fire,Flying,True,Fire,Fire,True,True
8397,Shadow Charizard,Fire Spin,Blast Burn,18.586,377.2,2421.3,Blast Burn,6.0,Fire,Flying,True,Fire,Fire,True,True
817,Shadow Arcanine,Fire Fang,Flamethrower,17.052,368.0,1824.6,Flamethrower,59.0,Fire,NaN,True,Fire,Fire,True,True


In [6]:
calcyFilename = 'calcyIV_history_20210316_110351.csv'

Note for later:
This lists the number of pokemon with similar attacks:
Result3 = calcy.groupby(['Pokemon',"#",'Fast move','Special move'  ,"Shadow"]).size()


In [8]:


moreData = match.calcyImportAndReturnTopNumbers(calcyFilename,allData)
moreData.head(5)

temp = moreData[moreData["ShadowForm"]==2]
temp.head(5)

calcyData = pd.read_csv(calcyFilename, encoding='utf8')
calcyData.head(5)

# read the csv files into dataframes
calcyData = pd.read_csv(calcyFilename, encoding='utf8')

# get the data we're interested in. Name and score
calcy = calcyData[["Name","Nr","Fast move","Special move","Special move 2","CP","Saved","Lucky","ShadowForm"]]

# drop those that are not saved:
calcy = calcy[calcy["Saved"]==1]

#TODO do something with Geodude Normal etc.
    
# rename shadow and XL to remove the quotation marks
#calcy['Shadow'] = calcy['Shadow'].str.replace(r'"', '')

# rename columns                                 
calcy = calcy.rename(columns={'Name': 'Pokemon'})
calcy = calcy.rename(columns={'Nr': '#'})

# drop any blank fast/special moves:
calcy = calcy[~calcy["Fast move"].str.startswith('-')]
calcy = calcy[~calcy["Fast move"].str.startswith(" -")]
calcy = calcy[~calcy["Special move"].str.startswith("-")]
calcy = calcy[~calcy["Special move"].str.startswith(' -')]

# rename - whitespace. Infuriating
calcy['Fast move'] = calcy['Fast move'].str.replace(r'-', ' ')
calcy['Special move'] = calcy['Special move'].str.replace(r'-', ' ')
calcy['Special move 2'] = calcy['Special move 2'].str.replace(r'-', ' ')

# move the "shadow" tag to the front rather than the back
calcyShadowFound = calcy["ShadowForm"]==2
calcy['Pokemon'] = calcy['Pokemon'].str.replace(r' Shadow', '')
calcy.loc[calcyShadowFound,'Pokemon'] = 'Shadow ' + calcy.loc[calcyShadowFound,'Pokemon'].astype(str)

# create shadow tag
calcy["Shadow"] = False
calcy.loc[calcyShadowFound,'Shadow'] = True

# merge : this needs to be a 2 step process
#Result1 = calcy.merge(setData,how='inner', on=['Pokemon',"#",'Fast move','Special move'  ,"Shadow"])
#Result2 = calcy.merge(setData,how='inner', on=['Pokemon',"#",'Fast move','Special move 2',"Shadow"])
Result1 = setData.merge(calcy,how='outer', on=['Pokemon',"#",'Fast move','Special move'  ,"Shadow"])
Result2 = setData.merge(calcy,how='outer', on=['Pokemon',"#",'Fast move','Special move 2',"Shadow"])

# rename columns
Result1 = Result1.rename(columns={'Special move 2_x': 'Special move 2'})
Result1 = Result1.rename(columns={'Special move 2_y': 'Temp'})
Result2 = Result2.rename(columns={'Special move_x': 'Special move'})
Result2 = Result2.rename(columns={'Special move_y': 'Temp'})

Result = pd.concat([Result1,Result2],sort=False)

# drop the column
Result.drop(columns=['Temp'], inplace=True)

# https://stackoverflow.com/questions/50293496/count-occurrences-of-item-in-one-dataframe-in-another

count_series = Result["Pokemon"].value_counts()

count_series.head(50)

Result.head(50)

,Pokemon,Fast move,Special move,DPS,TDO,DPS^3*TDO,Special move 2,#,Type1,Type2,Released_2021_03_13,TypeFast,TypeSpecial,Shadow,TopPvE,CP,Saved,Lucky,ShadowForm
0,Shadow Porygon-Z,Lock-On,Hyper Beam,17.528,330.8,1781.3,Hyper Beam,474.0,Normal,NaN,True,Normal,Normal,True,True,NaN,NaN,NaN,NaN
1,Porygon-Z,Lock-On,Return,15.343,347.5,1255.1,Return,474.0,Normal,NaN,True,Normal,Normal,False,True,NaN,NaN,NaN,NaN
2,Shadow Ambipom,Scratch,Hyper Beam,14.843,246.8,807.2,Hyper Beam,424.0,Normal,NaN,True,Normal,Normal,True,True,NaN,NaN,NaN,NaN
3,Shadow Porygon2,Lock-On,Hyper Beam,13.629,304.0,769.7,Hyper Beam,233.0,Normal,NaN,True,Normal,Normal,True,True,NaN,NaN,NaN,NaN
4,Ambipom,Scratch,Return,13.165,262.7,599.5,Return,424.0,Normal,NaN,True,Normal,Normal,False,True,NaN,NaN,NaN,NaN
5,Shadow Entei,Fire Fang,Overheat,18.379,488.3,3031.7,Overheat,244.0,Fire,NaN,True,Fire,Fire,True,True,NaN,NaN,NaN,NaN
6,Reshiram,Fire Fang,Overheat,17.409,557.8,2943.1,Overheat,643.0,Dragon,Fire,True,Fire,Fire,False,True,NaN,NaN,NaN,NaN
7,Shadow Moltres,Fire Spin,Overheat,18.796,439.2,2916.5,Overheat,146.0,Fire,Flying,True,Fire,Fire,True,True,NaN,NaN,NaN,NaN
8,Shadow Charizard,Fire Spin,Blast Burn,18.586,377.2,2421.3,Blast Burn,6.0,Fire,Flying,True,Fire,Fire,True,True,NaN,NaN,NaN,NaN
9,Shadow Arcanine,Fire Fang,Flamethrower,17.052,368.0,1824.6,Flamethrower,59.0,Fire,NaN,True,Fire,Fire,True,True,NaN,NaN,NaN,NaN


In [83]:
# This is for all data, but with any pokemon found that has either a fast, or special, or special2 move that is optimal (saved)

Result3 = setData.merge(calcy,how='outer', on=['Pokemon',"#",'Fast move'     ,"Shadow"])
Result4 = setData.merge(calcy,how='outer', on=['Pokemon',"#",'Special move'  ,"Shadow"])
Result5 = setData.merge(calcy,how='outer', on=['Pokemon',"#",'Special move 2',"Shadow"])

# rename columns
Result3 = Result3.rename(columns={'Special move_y': 'Temp'})
Result3 = Result3.rename(columns={'Special move 2_y': 'Temp2'})
Result3 = Result3.rename(columns={'Special move_x': 'Special move'})
Result3 = Result3.rename(columns={'Special move 2_x': 'Special move 2'})

Result4 = Result4.rename(columns={'Fast move_y': 'Temp'})
Result4 = Result4.rename(columns={'Fast move_x': 'Fast move'})
Result4 = Result4.rename(columns={'Special move 2_y': 'Temp2'})
Result4 = Result4.rename(columns={'Special move 2_x': 'Special move 2'})

Result5 = Result5.rename(columns={'Fast move_y': 'Temp'})
Result5 = Result5.rename(columns={'Fast move_x': 'Fast move'})
Result5 = Result5.rename(columns={'Special move_y': 'Temp2'})
Result5 = Result5.rename(columns={'Special move_x': 'Special move'})

ResultAny = pd.concat([Result3,Result4,Result5],sort=False)

# drop the column
ResultAny.drop(columns=['Temp'], inplace=True)
ResultAny.drop(columns=['Temp2'], inplace=True)

# ResultAny.head(40)

ResultJustPokemon = setData.merge(calcy,how='outer', on=['Pokemon',"#","Shadow"])
ResultJustPokemon = ResultJustPokemon.rename(columns={'Fast move_x': 'Fast move'})
ResultJustPokemon = ResultJustPokemon.rename(columns={'Special move_x': 'Special move'})
ResultJustPokemon = ResultJustPokemon.rename(columns={'Special move 2_x': 'Special move 2'})

ResultJustPokemon.head(40)



,Pokemon,Fast move,Special move,DPS,TDO,DPS^3*TDO,Special move 2,#,Type1,Type2,Released_2021_03_13,TypeFast,TypeSpecial,Shadow,TopPvE,FoundStr,FoundCount,Fast move_y,Special move_y,Special move 2_y,CP,Saved,Lucky,ShadowForm
0,Shadow Porygon-Z,Lock-On,Hyper Beam,17.528,330.8,1781.3,Hyper Beam,474.0,Normal,NaN,True,Normal,Normal,True,True,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Porygon-Z,Lock-On,Return,15.343,347.5,1255.1,Return,474.0,Normal,NaN,True,Normal,Normal,False,True,"Porygon-Z&CP2866,CP1381,CP2866,CP1381",4.0,Hidden Power Electric,Tri Attack,,2866.0,1.0,1.0,1.0
2,Porygon-Z,Lock-On,Return,15.343,347.5,1255.1,Return,474.0,Normal,NaN,True,Normal,Normal,False,True,"Porygon-Z&CP2866,CP1381,CP2866,CP1381",4.0,Lock On,Tri Attack,,1381.0,1.0,1.0,1.0
3,Shadow Ambipom,Scratch,Hyper Beam,14.843,246.8,807.2,Hyper Beam,424.0,Normal,NaN,True,Normal,Normal,True,True,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Shadow Porygon2,Lock-On,Hyper Beam,13.629,304.0,769.7,Hyper Beam,233.0,Normal,NaN,True,Normal,Normal,True,True,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Ambipom,Scratch,Return,13.165,262.7,599.5,Return,424.0,Normal,NaN,True,Normal,Normal,False,True,"Ambipom&CP2193,CP2193",2.0,Scratch,Hyper Beam,,2193.0,1.0,1.0,1.0
6,Shadow Entei,Fire Fang,Overheat,18.379,488.3,3031.7,Overheat,244.0,Fire,NaN,True,Fire,Fire,True,True,"Entei&shadow&CP740,CP740",2.0,Fire Fang,Iron Head,,740.0,1.0,1.0,2.0
7,Reshiram,Fire Fang,Overheat,17.409,557.8,2943.1,Overheat,643.0,Dragon,Fire,True,Fire,Fire,False,True,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Shadow Moltres,Fire Spin,Overheat,18.796,439.2,2916.5,Overheat,146.0,Fire,Flying,True,Fire,Fire,True,True,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Shadow Moltres,Wing Attack,Sky Attack,19.864,464.2,3638.2,Sky Attack,146.0,Fire,Flying,True,Flying,Flying,True,True,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# counting the data manually
setData["FoundCountExact"] = 0
setData["FoundCountAny"] = 0
setData["FoundCountJustPokemon"] = 0
setData["FoundStr"] = ""

for index, row in setData.iterrows():
    thisPokemon = row["Pokemon"]
    
    # count number of exact matches (fast and any special) in results db
    caseExact1 = Result['Pokemon'] == thisPokemon
    caseExact2 = Result['CP'] > 0
    caseExactBoth = caseExact1 & caseExact2
    # any move count
    caseAny1 = ResultAny['Pokemon'] == thisPokemon
    caseAny2 = ResultAny['CP'] > 0
    caseAnyBoth = caseAny1 & caseAny2
    #Just pokemon count
    caseJP1 = ResultJustPokemon['Pokemon'] == thisPokemon
    caseJP2 = ResultJustPokemon['CP'] > 0
    caseJPBoth = caseJP1 & caseJP2

    # convert this into a list we can copy/paste for pokemon go:
    listExactOut = Result.loc[caseExactBoth,'CP']
    listAnyOut = ResultAny.loc[caseAnyBoth,'CP']
    listJPOut = ResultJustPokemon.loc[caseJPBoth,'CP']

    typeStr = ["CP"+str(int(inte)) for inte in listExactOut]
    typeStr = ",".join(typeStr)
    # print(typeStr)
    # put this in the setData db again
    setIndex = setData["Pokemon"]==thisPokemon
    if len(listExactOut) == 0:
        setData.loc[setIndex,"FoundStr"] = ""
    elif thisPokemon.startswith('Shadow '):
        thisPokemon = thisPokemon.replace(r'Shadow ', '')   
        setData.loc[setIndex,"FoundStr"] = thisPokemon + "&shadow&" + typeStr
    else:
        setData.loc[setIndex,"FoundStr"] = thisPokemon + "&" + typeStr
    # how many we found
    setData.loc[setIndex,"FoundCountExact"] = len(listExactOut) 
    setData.loc[setIndex,"FoundCountAny"] = len(listAnyOut) 
    setData.loc[setIndex,"FoundCountJustPokemon"] = len(listJPOut) 

setData.head(10)

,Pokemon,Fast move,Special move,DPS,TDO,DPS^3*TDO,Special move 2,#,Type1,Type2,Released_2021_03_13,TypeFast,TypeSpecial,Shadow,TopPvE,FoundStr,FoundCount,FoundCountExact,FoundCountAny,FoundCountJustPokemon
674,Shadow Porygon-Z,Lock-On,Hyper Beam,17.528,330.8,1781.3,Hyper Beam,474.0,Normal,NaN,True,Normal,Normal,True,True,,0,0,0,0
2129,Porygon-Z,Lock-On,Return,15.343,347.5,1255.1,Return,474.0,Normal,NaN,True,Normal,Normal,False,True,"Porygon-Z&CP2866,CP1381,CP2866,CP1381",4,4,6,2
705,Shadow Ambipom,Scratch,Hyper Beam,14.843,246.8,807.2,Hyper Beam,424.0,Normal,NaN,True,Normal,Normal,True,True,,0,0,0,0
676,Shadow Porygon2,Lock-On,Hyper Beam,13.629,304.0,769.7,Hyper Beam,233.0,Normal,NaN,True,Normal,Normal,True,True,,0,0,0,0
2246,Ambipom,Scratch,Return,13.165,262.7,599.5,Return,424.0,Normal,NaN,True,Normal,Normal,False,True,"Ambipom&CP2193,CP2193",2,2,3,1
2284,Shadow Entei,Fire Fang,Overheat,18.379,488.3,3031.7,Overheat,244.0,Fire,NaN,True,Fire,Fire,True,True,"Entei&shadow&CP740,CP740",2,2,3,1
2285,Reshiram,Fire Fang,Overheat,17.409,557.8,2943.1,Overheat,643.0,Dragon,Fire,True,Fire,Fire,False,True,,0,0,0,0
2289,Shadow Moltres,Fire Spin,Overheat,18.796,439.2,2916.5,Overheat,146.0,Fire,Flying,True,Fire,Fire,True,True,,0,0,0,0
8397,Shadow Charizard,Fire Spin,Blast Burn,18.586,377.2,2421.3,Blast Burn,6.0,Fire,Flying,True,Fire,Fire,True,True,,0,0,0,0
817,Shadow Arcanine,Fire Fang,Flamethrower,17.052,368.0,1824.6,Flamethrower,59.0,Fire,NaN,True,Fire,Fire,True,True,,0,0,0,0
